## Import dependencies and load in the `.csv` and `.TSV000` files and create the 5 DataFrames

In [1]:
# Load dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings("ignore")

In [2]:
# Load in the state Redfin data
state_df = pd.read_csv("Resources/state_market_tracker.tsv000", sep="\t")

# Load in the national Redfin data
national_df = pd.read_csv("Resources/us_national_market_tracker.tsv000", sep="\t")

# Load in the `Home Price Index` Redfin data
price_index = pd.read_csv("Resources/National_RHPI_Index.csv", encoding="utf-16", sep="\t")

# Load in the `MoM HPI Index` Redfin data
mom_index = pd.read_csv("Resources/MoM.csv", encoding="utf-16", sep="\t")

# Load in 'https://fred.stlouisfed.org/series/MORTGAGE30US'
rates_df = pd.read_csv("Resources/MORTGAGE30US.csv")

## Clean up the `state_df` and `national_df` columns to align with "Date" for all of the Redfin data sources.

In [3]:
# Rename 'period_begin' to 'Date' 
state_df.rename(columns={"period_begin": "Date"}, inplace=True)

# Reformat to MM/DD/YY
state_df["Date"] = pd.to_datetime(state_df["Date"]).dt.strftime("%-m/%-d/%Y")

# Drop non-feature columns, including 'period_end'
columns_to_drop_state = ['period_end', 'region_type_id', 'state_code', 'property_type_id', 
                         'parent_metro_region', 'parent_metro_region_metro_code', 'last_updated']
state_df.drop(columns=columns_to_drop_state, inplace=True)

In [4]:
# Sort state_df by 'period_begin'
state_df = state_df.sort_values(by="Date", ascending=True)

# View the state_df
state_df.head()

,Date,period_duration,region_type,table_id,is_seasonally_adjusted,region,city,state,property_type,median_sale_price,...,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy
19971,1/1/2012,30,state,23,f,Oklahoma,NaN,Oklahoma,Townhouse,87200,...,-0.022731,0.000000,-0.181818,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000
10500,1/1/2012,30,state,10,f,New Hampshire,NaN,New Hampshire,All Residential,184000,...,0.004329,0.146886,-0.005915,0.010146,NaN,NaN,NaN,0.007093,0.000636,-0.000240
31826,1/1/2012,30,state,42,f,Virginia,NaN,Virginia,Multi-Family (2-4 Unit),40000,...,-0.037407,0.142857,0.142857,0.017857,NaN,NaN,NaN,0.000000,0.000000,0.000000
4385,1/1/2012,30,state,47,f,Michigan,NaN,Michigan,All Residential,79900,...,0.005616,0.188435,0.008916,0.011066,NaN,NaN,NaN,0.013045,-0.000528,0.000734
28684,1/1/2012,30,state,12,f,New Jersey,NaN,New Jersey,All Residential,247000,...,-0.007615,0.054863,-0.003108,-0.007739,0.000549,0.000502,NaN,0.007897,0.001832,0.006060


In [5]:
# Check for date range of DataFrame end dates of the state_df
first_date = state_df["Date"].min()
last_date = state_df["Date"].max()

# Display results
print(f"Beginning Date: {first_date}")
print(f"End Date: {last_date}")

Beginning Date: 1/1/2012
End Date: 9/1/2024


In [6]:
# Rename 'period_begin' to 'Date' 
national_df.rename(columns={"period_begin": "Date"}, inplace=True)

# Reformat to MM/DD/YY
national_df["Date"] = pd.to_datetime(national_df["Date"]).dt.strftime("%-m/%-d/%Y")

# Drop non-feature columns, including 'period_end'
columns_to_drop_national = ['period_end', 'region_type_id', 'state_code', 'property_type_id', 
                         'parent_metro_region', 'parent_metro_region_metro_code', 'last_updated']
national_df.drop(columns=columns_to_drop_national, inplace=True)

In [7]:
# Sort national_df by 'period_begin'
national_df = national_df.sort_values(by="Date", ascending=True)

# View the national_df
national_df.head()

,Date,period_duration,region_type,table_id,is_seasonally_adjusted,region,city,state,property_type,median_sale_price,...,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy
452,1/1/2012,30,national,1400,f,National,National,U.S.,Single Units Only,159562.000000,...,0.002164,0.173126,0.002568,0.004076,0.066841,0.012491,-0.011568,0.104603,0.027196,0.029610
642,1/1/2012,30,national,1400,t,National,National,U.S.,All Residential,170767.650108,...,NaN,0.199349,NaN,NaN,0.068864,NaN,NaN,0.119988,NaN,NaN
628,1/1/2012,30,national,1400,f,National,National,U.S.,Single Family Residential,164394.000000,...,0.002501,0.176420,0.002771,0.007864,0.065524,0.012231,-0.011690,0.104181,0.026586,0.029462
623,1/1/2012,30,national,1400,f,National,National,U.S.,Townhouse,150997.000000,...,0.002163,0.171398,0.003141,0.003095,0.069660,0.012516,-0.011709,0.100036,0.025608,0.026326
796,1/1/2012,30,national,1400,f,National,National,U.S.,Condo/Co-op,134728.000000,...,-0.000079,0.152783,0.001507,-0.019809,0.073374,0.014090,-0.010111,0.107596,0.029637,0.032152


## Create the HPI Index file by merging the monthly Index SA (Seasonally Adjusted) price index with the MoM (Month over  Month) changes for both Redfins HPI and Case Shillers price index.

In [8]:
# Update the column name to match Redfin website chart
price_index.rename(columns={"Unnamed: 1": "Index SA"}, inplace=True)

# View the price_index
price_index.head()

,Date,Index SA
0,1/1/2012,59.9
1,2/1/2012,60.4
2,3/1/2012,60.8
3,4/1/2012,61.2
4,5/1/2012,61.5


In [9]:
# Reorder columns in mom_index
new_order = ["Date", "Redfin HPI MoM", "Case Shiller Index MoM"]

mom_index = mom_index[new_order]

# Verify the new column order
mom_index.head()

,Date,Redfin HPI MoM,Case Shiller Index MoM
0,1/1/2012,0.00%,-0.00048
1,2/1/2012,0.00%,-0.00056
2,3/1/2012,0.00%,0.01004
3,4/1/2012,0.72%,0.00907
4,5/1/2012,0.62%,0.00719


In [10]:
# Combine the HPI and MoM data
hpi_index = pd.merge(price_index, mom_index, on="Date", how="inner")
hpi_index.head()

,Date,Index SA,Redfin HPI MoM,Case Shiller Index MoM
0,1/1/2012,59.9,0.00%,-0.00048
1,2/1/2012,60.4,0.00%,-0.00056
2,3/1/2012,60.8,0.00%,0.01004
3,4/1/2012,61.2,0.72%,0.00907
4,5/1/2012,61.5,0.62%,0.00719


In [11]:
# Check for date range of DataFrame end dates of the hpi_index
first = hpi_index["Date"].min()
last = hpi_index["Date"].max()

# Display results
print(f"Beginning Date: {first}")
print(f"End Date: {last}")

Beginning Date: 1/1/2012
End Date: 9/1/2024


## Check the data types structures and statistics for the 4 DataFrames to confirm which datasets to merge and where.

In [12]:
# Check the shape and data breakdown for hpi_index
hpi_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    158 non-null    object 
 1   Index SA                158 non-null    float64
 2   Redfin HPI MoM          158 non-null    object 
 3   Case Shiller Index MoM  158 non-null    float64
dtypes: float64(2), object(2)
memory usage: 5.1+ KB


In [16]:
# Rename the columns to be in line with hpi_index 
rates_df.rename(columns={"observation_date": "Date", "MORTGAGE30US": "30_Year_%"}, inplace=True)

# Convert the 'Date' column to a datetime object and reformat it to 'MM/DD/YYYY'
rates_df["Date"] = pd.to_datetime(rates_df["Date"]).dt.strftime("%-m/%-d/%Y")

# View rates_df
rates_df.tail()

,Date,30_Year_%
682,1/30/2025,6.95
683,2/6/2025,6.89
684,2/13/2025,6.87
685,2/20/2025,6.85
686,2/27/2025,6.76


In [14]:
# Check the shape and data breakdown for rates_df
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687 entries, 0 to 686
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       687 non-null    object 
 1   30_Year_%  687 non-null    float64
dtypes: float64(1), object(1)
memory usage: 10.9+ KB


In [17]:
# Check for date range of DataFrame end dates of the rates_df
print(rates_df["Date"].unique()) 

1/10/2013
9/9/2021
['1/5/2012' '1/12/2012' '1/19/2012' '1/26/2012' '2/2/2012' '2/9/2012'
 '2/16/2012' '2/23/2012' '3/1/2012' '3/8/2012' '3/15/2012' '3/22/2012'
 '3/29/2012' '4/5/2012' '4/12/2012' '4/19/2012' '4/26/2012' '5/3/2012'
 '5/10/2012' '5/17/2012' '5/24/2012' '5/31/2012' '6/7/2012' '6/14/2012'
 '6/21/2012' '6/28/2012' '7/5/2012' '7/12/2012' '7/19/2012' '7/26/2012'
 '8/2/2012' '8/9/2012' '8/16/2012' '8/23/2012' '8/30/2012' '9/6/2012'
 '9/13/2012' '9/20/2012' '9/27/2012' '10/4/2012' '10/11/2012' '10/18/2012'
 '10/25/2012' '11/1/2012' '11/8/2012' '11/15/2012' '11/21/2012'
 '11/29/2012' '12/6/2012' '12/13/2012' '12/20/2012' '12/27/2012'
 '1/3/2013' '1/10/2013' '1/17/2013' '1/24/2013' '1/31/2013' '2/7/2013'
 '2/14/2013' '2/21/2013' '2/28/2013' '3/7/2013' '3/14/2013' '3/21/2013'
 '3/28/2013' '4/4/2013' '4/11/2013' '4/18/2013' '4/25/2013' '5/2/2013'
 '5/9/2013' '5/16/2013' '5/23/2013' '5/30/2013' '6/6/2013' '6/13/2013'
 '6/20/2013' '6/27/2013' '7/3/2013' '7/11/2013' '7/18/2013' '7/25/

In [19]:
# Check the stats for state_df
state_df.describe()

,period_duration,table_id,city,median_sale_price,median_sale_price_mom,median_sale_price_yoy,median_list_price,median_list_price_mom,median_list_price_yoy,median_ppsf,...,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy
count,39054.0,39054.000000,0.0,3.905400e+04,39054.000000,39054.000000,3.894800e+04,38886.000000,38869.000000,38937.000000,...,37628.000000,39003.000000,39003.000000,39003.000000,30865.000000,30396.000000,28703.000000,38903.000000,38835.000000,38812.000000
mean,30.0,26.044272,NaN,2.742932e+05,0.021081,0.088713,3.038891e+05,0.008675,0.066152,172.388910,...,0.002766,0.208941,0.000308,0.008471,0.163705,0.001198,0.012557,0.252904,0.001960,0.025447
std,0.0,14.661014,NaN,1.657464e+05,0.256906,0.250174,1.645740e+05,0.076484,0.084074,282.322977,...,0.025963,0.142651,0.076999,0.102753,0.087796,0.047395,0.063022,0.195527,0.078571,0.109723
min,30.0,1.000000,NaN,7.000000e+03,-0.913000,-0.941347,5.300000e+04,-0.743884,-0.725075,3.000000,...,-0.539383,0.000000,-1.000000,-1.000000,0.000009,-0.888889,-0.800000,0.000000,-1.000000,-1.000000
25%,30.0,13.000000,NaN,1.686000e+05,-0.029909,0.014501,1.999000e+05,-0.027343,0.024691,101.000000,...,-0.003721,0.114130,-0.019844,-0.023244,0.103659,-0.017844,-0.016846,0.059043,-0.024598,-0.012868
50%,30.0,26.000000,NaN,2.302000e+05,0.004885,0.063599,2.600000e+05,0.001303,0.058138,138.000000,...,0.002735,0.181818,0.000000,0.005443,0.158801,0.002350,0.005715,0.248948,0.000000,0.009057
75%,30.0,39.000000,NaN,3.294750e+05,0.044059,0.127140,3.634000e+05,0.036497,0.098160,198.000000,...,0.009572,0.283181,0.020287,0.043603,0.218351,0.021613,0.035509,0.394383,0.025743,0.064291
max,30.0,51.000000,NaN,3.275000e+06,22.152709,12.571429,4.700000e+06,3.578644,3.630526,40906.000000,...,0.650629,1.000000,1.000000,1.000000,1.000000,0.645299,0.681287,1.000000,1.000000,1.000000


In [20]:
# Check the shape and data breakdown for state_df
state_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39054 entries, 19971 to 12650
Data columns (total 51 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Date                         39054 non-null  object 
 1   period_duration              39054 non-null  int64  
 2   region_type                  39054 non-null  object 
 3   table_id                     39054 non-null  int64  
 4   is_seasonally_adjusted       39054 non-null  object 
 5   region                       39054 non-null  object 
 6   city                         0 non-null      float64
 7   state                        39054 non-null  object 
 8   property_type                39054 non-null  object 
 9   median_sale_price            39054 non-null  int64  
 10  median_sale_price_mom        39054 non-null  float64
 11  median_sale_price_yoy        39054 non-null  float64
 12  median_list_price            38948 non-null  float64
 13  median_list_price

In [21]:
# Check the stats for national_df
national_df.describe()

,period_duration,table_id,median_sale_price,median_sale_price_mom,median_sale_price_yoy,median_list_price,median_list_price_mom,median_list_price_yoy,median_ppsf,median_ppsf_mom,...,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy
count,1738.0,1738.0,1738.000000,1733.000000,1678.000000,1738.000000,1733.000000,1678.000000,1738.000000,1733.000000,...,1678.000000,1738.000000,1733.000000,1678.000000,1738.000000,1733.000000,1678.000000,1738.000000,1733.000000,1678.000000
mean,30.0,1400.0,286493.388234,0.006178,0.074891,298560.925160,0.006763,0.071123,170.516200,0.006023,...,0.002367,0.268276,0.000483,0.008212,0.111060,0.000774,0.007476,0.331898,0.001665,0.019804
std,0.0,0.0,80656.713597,0.021647,0.050004,81834.461136,0.031055,0.041014,52.816614,0.019261,...,0.010424,0.088036,0.018612,0.079706,0.037923,0.018440,0.030579,0.126744,0.030051,0.073017
min,30.0,1400.0,130975.000000,-0.143796,-0.071733,148511.000000,-0.110785,-0.028655,67.000000,-0.122746,...,-0.038200,0.140760,-0.099338,-0.282408,0.037674,-0.200867,-0.216351,0.100036,-0.116967,-0.258460
25%,30.0,1400.0,221433.250000,-0.004585,0.045457,233638.061937,-0.008107,0.041776,129.019677,-0.003144,...,-0.001758,0.210687,-0.008016,-0.024453,0.082755,-0.003842,-0.010829,0.228934,-0.014392,-0.006290
50%,30.0,1400.0,270385.673069,0.005583,0.066117,283509.226120,0.004287,0.063884,157.000000,0.005065,...,0.002871,0.239732,-0.000202,0.013924,0.109658,0.001582,0.003528,0.334078,-0.000737,0.028485
75%,30.0,1400.0,352252.215293,0.014934,0.100633,362129.924387,0.014243,0.096644,214.830888,0.014720,...,0.005892,0.294808,0.008204,0.031742,0.136298,0.007702,0.024695,0.407688,0.014049,0.054311
max,30.0,1400.0,480069.000000,0.119120,0.291577,550229.000000,0.177777,0.228002,319.653928,0.152847,...,0.042060,0.602377,0.072822,0.312275,0.322287,0.201428,0.199815,0.688481,0.101049,0.281640


In [22]:
# Check the shape and data breakdown for national_df
national_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1738 entries, 452 to 1403
Data columns (total 51 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Date                         1738 non-null   object 
 1   period_duration              1738 non-null   int64  
 2   region_type                  1738 non-null   object 
 3   table_id                     1738 non-null   int64  
 4   is_seasonally_adjusted       1738 non-null   object 
 5   region                       1738 non-null   object 
 6   city                         1738 non-null   object 
 7   state                        1738 non-null   object 
 8   property_type                1738 non-null   object 
 9   median_sale_price            1738 non-null   float64
 10  median_sale_price_mom        1733 non-null   float64
 11  median_sale_price_yoy        1678 non-null   float64
 12  median_list_price            1738 non-null   float64
 13  median_list_price_mom

## Create copies of the 4 DataFrames, setting aside the originals for comparison and plotting. Merge the copied versions `state_df_copy` and `hpi_index_copy`. 

In [23]:
# Create copies of the DataFrames
state_df_copy = state_df.copy()
national_df_copy = national_df.copy()
hpi_index_copy = hpi_index.copy()
rates_df_copy = rates_df.copy()


In [24]:
# Combine state and hpi DataFrames
combined_df = pd.merge(state_df_copy, hpi_index_copy, on="Date", how="inner")

# Reorder columns in combined_df
column_order = ["Date"] + hpi_index_copy.columns.tolist()[1:] + state_df_copy.columns.tolist()[1:]
combined_df = combined_df[column_order]

# Verify the results
combined_df.head()

,Date,Index SA,Redfin HPI MoM,Case Shiller Index MoM,period_duration,region_type,table_id,is_seasonally_adjusted,region,city,...,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy
0,1/1/2012,59.9,0.00%,-0.00048,30,state,23,f,Oklahoma,NaN,...,-0.022731,0.000000,-0.181818,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000
1,1/1/2012,59.9,0.00%,-0.00048,30,state,10,f,New Hampshire,NaN,...,0.004329,0.146886,-0.005915,0.010146,NaN,NaN,NaN,0.007093,0.000636,-0.000240
2,1/1/2012,59.9,0.00%,-0.00048,30,state,42,f,Virginia,NaN,...,-0.037407,0.142857,0.142857,0.017857,NaN,NaN,NaN,0.000000,0.000000,0.000000
3,1/1/2012,59.9,0.00%,-0.00048,30,state,47,f,Michigan,NaN,...,0.005616,0.188435,0.008916,0.011066,NaN,NaN,NaN,0.013045,-0.000528,0.000734
4,1/1/2012,59.9,0.00%,-0.00048,30,state,12,f,New Jersey,NaN,...,-0.007615,0.054863,-0.003108,-0.007739,0.000549,0.000502,NaN,0.007897,0.001832,0.006060


## Update the `rates_df` date and average the the interest rates. Reason is the start date would require back filling as we have 1/1/2012 (monthly) `combined_df`and 1/5/2012 as our start date for `rates_df`(off weekly). This adds in the interest rate feature to our combined data set. 

**Example, the first month is (3.91 + 3.89 + 3.88 + 3.98) / 4 = 3.915.**

In [25]:
# Convert 'Date' in rates_df_copy to datetime for easier processing
rates_df_copy["Date"] = pd.to_datetime(rates_df_copy["Date"])

# Extract year and month from weekly dates (Makes it a Year-Month format)
rates_df_copy["Month"] = rates_df_copy["Date"].dt.to_period("M")  

# Aggregate by month (Average 30-Year rate for each month)
rates_monthly = rates_df_copy.groupby("Month").agg({"30_Year_%": "mean"}).reset_index()

# Confirm data types
rates_monthly.dtypes

Month        period[M]
30_Year_%      float64
dtype: object

In [26]:
rates_monthly.head()

,Month,30_Year_%
0,2012-01,3.915
1,2012-02,3.890
2,2012-03,3.954
3,2012-04,3.910
4,2012-05,3.798


In [27]:
rates_monthly.dtypes

Month        period[M]
30_Year_%      float64
dtype: object

In [29]:
# Convert 'Month' (period[M]) to datetime format
rates_monthly["Date"] = rates_monthly["Month"].dt.to_timestamp()

# Format 'Date' to MM/DD/YYYY
rates_monthly["Date"] = rates_monthly["Date"].dt.strftime("%-m/%-d/%Y")

In [30]:
# Re-confirm the rates_monthly data types 
rates_monthly.dtypes

Month        period[M]
30_Year_%      float64
Date            object
dtype: object

In [32]:
# Drop the 'Month' column
rates_monthly.drop(columns=["Month"], inplace=True)

# Verify the results
rates_monthly.head()

,30_Year_%,Date
0,3.915,1/1/2012
1,3.890,2/1/2012
2,3.954,3/1/2012
3,3.910,4/1/2012
4,3.798,5/1/2012


In [33]:
# Merge with combined_df
complete_df = pd.merge(combined_df, rates_monthly, on="Date", how="inner")

# Verify the result
complete_df.head()

,Date,Index SA,Redfin HPI MoM,Case Shiller Index MoM,period_duration,region_type,table_id,is_seasonally_adjusted,region,city,...,sold_above_list,sold_above_list_mom,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,30_Year_%
0,1/1/2012,59.9,0.00%,-0.00048,30,state,23,f,Oklahoma,NaN,...,0.000000,-0.181818,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,3.915
1,1/1/2012,59.9,0.00%,-0.00048,30,state,10,f,New Hampshire,NaN,...,0.146886,-0.005915,0.010146,NaN,NaN,NaN,0.007093,0.000636,-0.000240,3.915
2,1/1/2012,59.9,0.00%,-0.00048,30,state,42,f,Virginia,NaN,...,0.142857,0.142857,0.017857,NaN,NaN,NaN,0.000000,0.000000,0.000000,3.915
3,1/1/2012,59.9,0.00%,-0.00048,30,state,47,f,Michigan,NaN,...,0.188435,0.008916,0.011066,NaN,NaN,NaN,0.013045,-0.000528,0.000734,3.915
4,1/1/2012,59.9,0.00%,-0.00048,30,state,12,f,New Jersey,NaN,...,0.054863,-0.003108,-0.007739,0.000549,0.000502,NaN,0.007897,0.001832,0.006060,3.915


In [34]:
complete_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39054 entries, 0 to 39053
Data columns (total 55 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Date                         39054 non-null  object 
 1   Index SA                     39054 non-null  float64
 2   Redfin HPI MoM               39054 non-null  object 
 3   Case Shiller Index MoM       39054 non-null  float64
 4   period_duration              39054 non-null  int64  
 5   region_type                  39054 non-null  object 
 6   table_id                     39054 non-null  int64  
 7   is_seasonally_adjusted       39054 non-null  object 
 8   region                       39054 non-null  object 
 9   city                         0 non-null      float64
 10  state                        39054 non-null  object 
 11  property_type                39054 non-null  object 
 12  median_sale_price            39054 non-null  int64  
 13  median_sale_pric

In [35]:
# Find the minimum and maximum dates
min_date = complete_df["Date"].min()
max_date = complete_df["Date"].max()

# Print the results
print(f"Minimum Date: {min_date}")
print(f"Maximum Date: {max_date}")

Minimum Date: 1/1/2012
Maximum Date: 9/1/2024


## Address all rows with missing values > 0.50%.

In [38]:
# Remove 'city' column as no values exist
complete_df.drop(columns=["city"], inplace=True)

# Update the Missing Price Drops with 0 as reference no price drops during that month from that region
complete_df["price_drops"].fillna(0, inplace=True)
complete_df["price_drops_mom"].fillna(0, inplace=True)
complete_df["price_drops_yoy"].fillna(0, inplace=True)

# Create new columns to flag instance where price drops are blanks - creates a feauture 
complete_df["price_drops_is_blank"] = complete_df["price_drops"] == 0
complete_df["price_drops_mom_is_blank"] = complete_df["price_drops_mom"] == 0
complete_df["price_drops_yoy_is_blank"] = complete_df["price_drops_yoy"] == 0

# Use Mean for missing row values for the Average List to Sale Prices 
complete_df["avg_sale_to_list"].fillna(complete_df["avg_sale_to_list"].mean(), inplace=True)
complete_df["avg_sale_to_list_mom"].fillna(complete_df["avg_sale_to_list_mom"].mean(), inplace=True)
complete_df["avg_sale_to_list_yoy"].fillna(complete_df["avg_sale_to_list_yoy"].mean(), inplace=True)


In [41]:
# Fill missing values with 0 for the specified columns
complete_df["off_market_in_two_weeks"].fillna(0, inplace=True)
complete_df["off_market_in_two_weeks_mom"].fillna(0, inplace=True)
complete_df["off_market_in_two_weeks_yoy"].fillna(0, inplace=True)

# Confirm the changes
print(complete_df[["off_market_in_two_weeks", "off_market_in_two_weeks_mom", "off_market_in_two_weeks_yoy"]].isnull().sum())


off_market_in_two_weeks        0
off_market_in_two_weeks_mom    0
off_market_in_two_weeks_yoy    0
dtype: int64


In [43]:
# Fill missing values in median-related columns with their column averages
columns_to_fill = [
    "median_list_price", "median_list_price_mom", "median_list_price_yoy",
    "median_ppsf", "median_ppsf_mom", "median_ppsf_yoy",
    "median_list_ppsf", "median_list_ppsf_mom", "median_list_ppsf_yoy"
]

# Apply mean imputation
for column in columns_to_fill:
    complete_df[column].fillna(complete_df[column].mean(), inplace=True)

# Verify the changes
print(complete_df[columns_to_fill].isnull().sum()) 

median_list_price        0
median_list_price_mom    0
median_list_price_yoy    0
median_ppsf              0
median_ppsf_mom          0
median_ppsf_yoy          0
median_list_ppsf         0
median_list_ppsf_mom     0
median_list_ppsf_yoy     0
dtype: int64


In [44]:
# Determine the percentage of null values present
missing_percent = complete_df.isnull().mean() * 100
print(missing_percent)

Date                           0.000000
Index SA                       0.000000
Redfin HPI MoM                 0.000000
Case Shiller Index MoM         0.000000
period_duration                0.000000
region_type                    0.000000
table_id                       0.000000
is_seasonally_adjusted         0.000000
region                         0.000000
state                          0.000000
property_type                  0.000000
median_sale_price              0.000000
median_sale_price_mom          0.000000
median_sale_price_yoy          0.000000
median_list_price              0.000000
median_list_price_mom          0.000000
median_list_price_yoy          0.000000
median_ppsf                    0.000000
median_ppsf_mom                0.000000
median_ppsf_yoy                0.000000
median_list_ppsf               0.000000
median_list_ppsf_mom           0.000000
median_list_ppsf_yoy           0.000000
homes_sold                     0.000000
homes_sold_mom                 0.000000


In [45]:
# Save the DataFrame as a CSV file
complete_df.to_csv("Resources/complete_data.csv", index=False)

# Print confirmation
print("CSV file has been saved successfully as 'complete_data.csv'")

CSV file has been saved successfully as 'complete_data.csv'
